In [1]:
from SVDModel import SVDModel
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

np.random.seed(3362)

In [2]:
# Parameters
PARAM = dict()
PARAM['path'] = {
  'ml':'./data/MovieLens25M/',
  'netflix':'./data/NetflixPrize/'
}
PARAM['features'] = [5,10,50,100,150,200,250,300]
PARAM['lr'] = [1e-1,1e-2,1e-3,1e-4,1e-5]
PARAM['epochs'] = [101,201,301]
PARAM['weight_decay'] = [0.02]
PARAM['stopping'] = [0.001]

PARAM

{'path': {'ml': './data/MovieLens25M/', 'netflix': './data/NetflixPrize/'},
 'features': [5, 10, 50, 100, 150, 200, 250, 300],
 'lr': [0.1, 0.01, 0.001, 0.0001, 1e-05],
 'epochs': [101, 201, 301],
 'weight_decay': [0.02],
 'stopping': [0.001]}

In [3]:
# ML_25M = {
#   'ratings': pd.read_csv(PARAM['path']['ml']+'ratings.csv', dtype={
#     'userId': np.int32,
#     'movieId': np.int32,
#     'rating': np.float64,
#     'timestamp':np.float64}).sample(2**16),
#   'movies': pd.read_csv(PARAM['path']['ml']+'movies.csv', dtype= {
#     'movieId':np.int32,
#     'title': str,
#     'genres': str
#   })
# }
# ML_25M['m_movies'] = ML_25M['movies']['movieId'].nunique()
# ML_25M['n_users'] = ML_25M['ratings']['userId'].nunique()
# ML_25M['n_ratings'] = len(ML_25M['ratings'])


# ML_25M['movies'].insert(1, 'newMovieId', value=range(1,len(ML_25M['movies'])+1))
# newMovieIdDict = {}
# for index, movie in ML_25M['movies'].iterrows():
#   newMovieIdDict[movie['movieId']] = movie['newMovieId']
# ML_25M['ratings'].replace({"movieId": newMovieIdDict}, inplace=True)
# ML_25M['ratings']['rating'] = ML_25M['ratings']['rating'].astype(np.float16)

# ML_25M

In [4]:
# print(ML_25M['ratings'].size)
# print(ML_25M['movies'].size)

In [3]:
names = ['userId','rating','timestamp','movieId']
df_netflix = pd.read_csv(PARAM['path']['netflix']+'data_1.txt', names=names, low_memory=False)
df_netflix_2 = pd.read_csv(PARAM['path']['netflix']+'data_2.txt', names=names, low_memory=False)
df_netflix_3 = pd.read_csv(PARAM['path']['netflix']+'data_3.txt', names=names, low_memory=False)
df_netflix_4 = pd.read_csv(PARAM['path']['netflix']+'data_4.txt', names=names, low_memory=False)
df_netflix = pd.concat([df_netflix,df_netflix_2,df_netflix_3,df_netflix_4], names=names)
df_netflix['movieId'] = df_netflix['movieId'].fillna(method='ffill')
df_netflix = df_netflix.dropna()
df_netflix['rating'] = df_netflix['rating'].astype(np.float64)
df_netflix['movieId'] = df_netflix['movieId'].astype(np.int16)
df_netflix['userId'] = df_netflix['userId'].astype(np.int32)
df_netflix['timestamp'] = pd.to_datetime(df_netflix['timestamp']).view(dtype=np.int64) // 10 ** 9

NETFLIX = {
  'ratings': df_netflix[['userId','movieId','rating','timestamp']].sample(2**16),
  'movies': pd.read_csv(PARAM['path']['netflix']+'movie_titles.csv',encoding='latin-1',names=['movieId','release_year','title']),
  'm_movies': df_netflix['movieId'].nunique(),
  'n_users': df_netflix['userId'].nunique(),
  'n_ratings': len(df_netflix)
}

In [6]:
# print(NETFLIX['ratings'].size)
# print(NETFLIX['movies'].size)

262144
53310


In [8]:
# svd.data[['userId','movieId','rating']].pivot_table(columns='movieId', index='userId', values='rating')

TypeError: No matching signature found

In [7]:
features = 5
lr = 0.01
epochs = 101
weight_decay = 0.02
stopping = 0.001

svd = SVDModel(features=features, lr=lr, epochs=epochs, weight_decay=weight_decay, stopping=stopping)
svd.data_loader(data=NETFLIX['ratings'], n_items = NETFLIX['m_movies'], n_users=NETFLIX['n_users'])
svd.split(0.6,0.8)


KeyboardInterrupt: 

In [ ]:
resuls_netflix = svd.training()
x = list(range(len(resuls_netflix[0])))
k=0

sns.set(rc={'figure.figsize':(12,8)})
sns.set(font_scale = 1.5)

plt.plot(x[-k:], resuls_netflix[0][-k:], 'r', label="Train")
plt.plot(x[-k:], resuls_netflix[1][-k:], 'g', label="Validation")
plt.title('Learning curves')
plt.xlabel('Epoch')
plt.ylabel('MSE')
leg = plt.legend(loc='best', shadow=True, fancybox=True)